In [1]:
import faiss

In [2]:
import numpy as np 

In [3]:
Xt = np.loadtxt('saved_db_100k.csv', delimiter = ',')

In [4]:
data = Xt[:,1:]
data[:1,:]

array([[0.8922525 , 0.2800759 , 0.26163924, 0.46114665, 0.894992  ,
        0.12171966, 0.24928057, 0.52260834, 0.24156952, 0.40916836,
        0.34428424, 0.07163954, 0.03442818, 0.09896666, 0.66252285,
        0.9862883 , 0.92981035, 0.69406784, 0.92269063, 0.44868845,
        0.55067986, 0.6401768 , 0.40450466, 0.2704888 , 0.7013119 ,
        0.30187392, 0.5553066 , 0.07334203, 0.2125904 , 0.05258554,
        0.61584634, 0.8074751 , 0.8789867 , 0.8133053 , 0.92158616,
        0.00497156, 0.7687784 , 0.3300553 , 0.6475257 , 0.08908683,
        0.21998852, 0.4679469 , 0.2687173 , 0.61768943, 0.90623456,
        0.55104804, 0.9345592 , 0.13406181, 0.4424578 , 0.7024428 ,
        0.15089315, 0.6129694 , 0.40907687, 0.46742535, 0.5311658 ,
        0.29712713, 0.7017058 , 0.91839385, 0.13668495, 0.08343965,
        0.2610187 , 0.14476323, 0.39879185, 0.37857074, 0.23424745,
        0.25922382, 0.30125505, 0.6162096 , 0.8325671 , 0.6945543 ]])

## Trial 1 : Using IndexFlatL2 as coarse quantizer and IVFPQ 

In [5]:
quantizer = faiss.IndexFlatL2(70)


In [6]:
index = faiss.IndexIVFPQ(quantizer,70,150,10,8)

In [7]:
index.train(data)

In [8]:
index.add(data)

In [9]:
Q = np.random.random((1,70))
Q

array([[0.15381216, 0.66812644, 0.31372989, 0.10027266, 0.71392351,
        0.47918202, 0.79061761, 0.2839595 , 0.73835129, 0.33685163,
        0.26239115, 0.48564119, 0.7535227 , 0.30939925, 0.94452713,
        0.6108156 , 0.41925184, 0.11893152, 0.64021558, 0.80980222,
        0.64452246, 0.36290637, 0.06519896, 0.12705238, 0.63469259,
        0.04459951, 0.69774183, 0.99725009, 0.53557804, 0.03400312,
        0.24559993, 0.69949419, 0.1097261 , 0.49857644, 0.98639776,
        0.80085001, 0.4395579 , 0.75673766, 0.94979435, 0.71310826,
        0.33734678, 0.14089238, 0.4325938 , 0.82813925, 0.93850437,
        0.60607363, 0.09464134, 0.51292329, 0.52612938, 0.37735626,
        0.43955997, 0.6982673 , 0.5049014 , 0.66387966, 0.75477134,
        0.0380632 , 0.99803922, 0.67927369, 0.2538282 , 0.61943442,
        0.60171119, 0.00256892, 0.3266489 , 0.40413375, 0.37117956,
        0.39109161, 0.29891015, 0.16337981, 0.08474247, 0.72612291]])

In [33]:
%%time

dist_x , id_x = index.search(Q,5)

dist_x

CPU times: total: 0 ns
Wall time: 2.13 ms


array([[5.275448 , 5.438001 , 5.5207634, 5.5406556, 5.6021733]],
      dtype=float32)

## Trial 2 : Using hnsw as a quantizer with IVFPQ

In [12]:
quantizer_hnsw = faiss.IndexHNSWFlat(70,10)
index_hnswivfpq = faiss.IndexIVFPQ(quantizer_hnsw, 70, 250, 10, 8)

In [13]:
%%time
index_hnswivfpq.train(data)
index_hnswivfpq.add(data)


CPU times: total: 1min 31s
Wall time: 14.2 s


In [34]:
%time dists , ids = index_hnswivfpq.search(Q,5)

dists

CPU times: total: 0 ns
Wall time: 2.52 ms


array([[5.8153067, 6.212041 , 6.2536707, 6.2595177, 6.3077292]],
      dtype=float32)

In [15]:
Q

array([[0.15381216, 0.66812644, 0.31372989, 0.10027266, 0.71392351,
        0.47918202, 0.79061761, 0.2839595 , 0.73835129, 0.33685163,
        0.26239115, 0.48564119, 0.7535227 , 0.30939925, 0.94452713,
        0.6108156 , 0.41925184, 0.11893152, 0.64021558, 0.80980222,
        0.64452246, 0.36290637, 0.06519896, 0.12705238, 0.63469259,
        0.04459951, 0.69774183, 0.99725009, 0.53557804, 0.03400312,
        0.24559993, 0.69949419, 0.1097261 , 0.49857644, 0.98639776,
        0.80085001, 0.4395579 , 0.75673766, 0.94979435, 0.71310826,
        0.33734678, 0.14089238, 0.4325938 , 0.82813925, 0.93850437,
        0.60607363, 0.09464134, 0.51292329, 0.52612938, 0.37735626,
        0.43955997, 0.6982673 , 0.5049014 , 0.66387966, 0.75477134,
        0.0380632 , 0.99803922, 0.67927369, 0.2538282 , 0.61943442,
        0.60171119, 0.00256892, 0.3266489 , 0.40413375, 0.37117956,
        0.39109161, 0.29891015, 0.16337981, 0.08474247, 0.72612291]])

## Statistics and Comparison between cosine similarity of Q and the vectors in data  

### Brute-Force cos-similarity 

In [17]:
b = Q[0]
for idx in range(90000):
    a = data[idx,:].reshape(1,-1)
    xos = np.dot(a, b)/(np.linalg.norm(a)*(np.linalg.norm(b)))
    if xos > 0.85:
        print(xos, idx)

[0.86028046] 484
[0.86244727] 1659
[0.85267913] 1955
[0.86971393] 4356
[0.85119305] 4770
[0.85463432] 4905
[0.85471078] 5188
[0.85385309] 5309
[0.85014784] 5962
[0.85541129] 6536
[0.85087479] 6664
[0.85235284] 7053
[0.85116026] 10620
[0.85683533] 11214
[0.85900165] 11350
[0.85552451] 12732
[0.85164718] 14396
[0.85175008] 14642
[0.86041158] 16535
[0.85152621] 16925
[0.85616794] 18265
[0.85322591] 18364
[0.8551893] 19170
[0.85164361] 19608
[0.85076729] 20653
[0.85423353] 23430
[0.86713062] 24887
[0.8522858] 25797
[0.85602785] 26134
[0.86167693] 26279
[0.8515973] 26284
[0.87781698] 26502
[0.86785472] 28026
[0.87271722] 29302
[0.86962894] 30724
[0.85324389] 33748
[0.8663655] 34764
[0.85157781] 35142
[0.86144645] 36637
[0.85032632] 37601
[0.8576791] 40497
[0.85479443] 42388
[0.8590084] 43262
[0.85477863] 43648
[0.86039308] 44251
[0.85914642] 44265
[0.85469947] 44998
[0.85351996] 45570
[0.86548895] 45616
[0.85395126] 46069
[0.87415312] 46474
[0.85860464] 47045
[0.8509762] 47241
[0.87849865] 

## Ids extracted from faiss hnswivfpq as nearest distance

In [31]:
print(id_x) #=> ivfpq no hnsw
b = Q[0]
for idx in id_x[0]:
    a = data[idx,:].reshape(1,-1)
    print(np.dot(a, b)/(np.linalg.norm(a)*(np.linalg.norm(b))))

[[35465 12536 59713  6664 86038]]
[0.84667727]
[0.84664095]
[0.85155206]
[0.85087479]
[0.81983764]


In [32]:
print(ids) #=> with hnsw

b = Q[0]
for idx in ids[0]:
    a = data[idx,:].reshape(1,-1)
    print(np.dot(a, b)/(np.linalg.norm(a)*(np.linalg.norm(b))))

[[34790 29302 63041 25871 79853]]
[0.83758056]
[0.87271722]
[0.85914947]
[0.83687283]
[0.85010479]


## PQ

In [20]:
import nanopq

In [21]:
N, Nt, D = 10000, 2000, 70
# X = np.random.random((N, D)).astype(np.float32)  # 10,000 128-dim vectors to be indexed
Xt = np.random.random((Nt, D)).astype(np.float32)  # 2,000 128-dim vectors for training
query = np.random.random((D,)).astype(np.float32)  # a 128-dim query vector

# Instantiate with M=8 sub-spaces
pq = nanopq.PQ(M=7)

# Train codewords
pq.fit(Xt)

# Encode to PQ-codes
X_code = pq.encode(data.astype(np.float32))  # (90000, 7) with dtype=np.uint8

# Results: create a distance table online, and compute Asymmetric Distance to each PQ-code 
dists = pq.dtable(query).adist(X_code)  # (90000, ) 

M: 7, Ks: 256, metric : <class 'numpy.uint8'>, code_dtype: l2
iter: 20, seed: 123
Training the subspace: 0 / 7
Training the subspace: 1 / 7
Training the subspace: 2 / 7
Training the subspace: 3 / 7
Training the subspace: 4 / 7
Training the subspace: 5 / 7
Training the subspace: 6 / 7
Encoding the subspace: 0 / 7
Encoding the subspace: 1 / 7
Encoding the subspace: 2 / 7
Encoding the subspace: 3 / 7
Encoding the subspace: 4 / 7
Encoding the subspace: 5 / 7
Encoding the subspace: 6 / 7


In [22]:
dists

array([ 8.315259 , 10.058419 ,  8.121955 , ..., 10.368339 ,  6.5310926,
        9.235798 ], dtype=float32)

## IVF_Flat 

In [23]:
# Generating random data (1M vectors with dimension 70)
np.random.seed(42)
Xt = np.random.rand(2000, 70).astype('float32')

# Parameters
d = 70  # Dimension of the vectors
nlist = 100  # Number of clusters
quantizer = faiss.IndexFlatL2(d)  # Flat index for clustering

# Create IndexIVFFlat
index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)

# Train the index
index.train(Xt)
index.add(data)

dists_flat , ids_flat = index.search(Q,5)

# Save the index
# faiss.write_index(index, "index_ivf_flat.index")


In [30]:
dists_flat

array([[5.3858967, 6.0609307, 6.4695992, 6.737144 , 6.7718644]],
      dtype=float32)

In [24]:
b = Q[0]
for idx in ids_flat[0]:
    a = data[idx,:].reshape(1,-1)
    print(np.dot(a, b)/(np.linalg.norm(a)*(np.linalg.norm(b))))

[0.87566758]
[0.85286316]
[0.84701471]
[0.85537844]
[0.83242422]


## Index IVFPQ only

In [26]:
# Generating random data (1M vectors with dimension 70)
np.random.seed(42)
Xt = np.random.rand(2000, 70).astype('float32')

# Parameters
d = 70  # Dimension of the vectors
nlist = 100  # Number of clusters
m = 7  # Number of subquantizers
nbits = 8  # Number of bits for quantization

# Create IndexIVFPQ
index = faiss.IndexIVFPQ(faiss.IndexFlatL2(d), d, nlist, m, nbits)

# Train the index
index.train(Xt)
index.add(data)

dists_ivfpq , ids_ivfpq = index.search(Q,5)



# Save the index
#faiss.write_index(index, "index_ivf_pq.index")


In [29]:
dists_ivfpq

array([[5.275448 , 5.438001 , 5.5207634, 5.5406556, 5.6021733]],
      dtype=float32)

In [27]:
b = Q[0]
for idx in ids_ivfpq[0]:
    a = data[idx,:].reshape(1,-1)
    print(np.dot(a, b)/(np.linalg.norm(a)*(np.linalg.norm(b))))

[0.87566758]
[0.80307938]
[0.84244655]
[0.85267913]
[0.82478373]


## IVF only 

In [28]:
# Generating random data (1M vectors with dimension 70)
np.random.seed(42)
Xt = np.random.rand(2000, 70).astype('float32')

# Parameters
d = 70  # Dimension of the vectors
nlist = 100  # Number of clusters
quantizer = faiss.IndexFlatL2(d)  # Flat index for clustering

# Create IndexIVF
index = faiss.IndexIVF(quantizer, d, nlist, faiss.METRIC_L2)

# Train the index
index.train(Xt)
index.add(data)

# Save the index
# faiss.write_index(index, "index_ivf.index")


AttributeError: No constructor defined - class is abstract